<a href="https://colab.research.google.com/github/zhestyatsky/MCL-WiC/blob/main/experiments/cosine_similarity_relu_with_threshold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install --upgrade pytorch-lightning==1.1.6 scikit-learn transformers==4.2.2 sentencepiece

     |████████████████████████████████| 696kB 22.8MB/s 
     |████████████████████████████████| 22.2MB 1.4MB/s 
     |████████████████████████████████| 1.8MB 59.2MB/s 
     |████████████████████████████████| 1.2MB 55.4MB/s 
     |████████████████████████████████| 276kB 63.9MB/s 
     |████████████████████████████████| 829kB 45.4MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 890kB 56.9MB/s 
     |████████████████████████████████| 2.9MB 52.8MB/s 
     |████████████████████████████████| 1.3MB 49.2MB/s 
     |████████████████████████████████| 143kB 60.5MB/s 
     |████████████████████████████████| 296kB 60.3MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=9f59123653234c0779074a8de312073d0471337797d3fba1769655a9bbf00811
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none

In [3]:
!git clone https://github.com/zhestyatsky/MCL-WiC.git MCL-WiC

import sys
sys.path.insert(0, '/content/MCL-WiC')

Cloning into 'MCL-WiC'...
remote: Enumerating objects: 543, done.
remote: Counting objects: 100% (543/543), done.
remote: Compressing objects: 100% (354/354), done.
remote: Total 543 (delta 269), reused 415 (delta 158), pack-reused 0
Receiving objects: 100% (543/543), 5.12 MiB | 23.62 MiB/s, done.
Resolving deltas: 100% (269/269), done.


In [4]:
### THIS IS WHERE YOU SELECT THE MODEL ###
### ALL AVAILABLE MODELS ARE LISTED IN src/training/config.py ###

MODEL_DESCRIPTION = "bert-large-cos_sim-relu"

In [5]:
from src.data.dataset import get_train_val_test_loaders
from src.training.models import get_model
from src.training.trainer import get_trainer
from src.util.util import seed_everything

seed_everything(42)

train_loader, val_loader, test_loader = get_train_val_test_loaders(MODEL_DESCRIPTION)
clf = get_model(MODEL_DESCRIPTION)
trainer = get_trainer()

trainer.fit(clf, train_loader, val_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | model          | BertModel        | 333 M 
1 | loss           | BCELoss          | 0     
2 | valid_accuracy | Accuracy         | 0     
3 | test_accuracy  | Accuracy         | 0     
4 | activation     | ReLU             | 0     
5 | cos            | CosineSimilarity | 0     
----------------------------------------------------
333 M     Trainable params
0         Non-trainable params
333 M     Total params


1

In [6]:
trainer.test(clf, val_loader)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [7]:
from src.training.hyperparams import find_threshold_if_needed

threshold, validation_accuracy = find_threshold_if_needed(val_loader.dataset.labels, clf.predictions_proba, MODEL_DESCRIPTION)
trainer.test(clf, test_loader)

Threshold: 0.5194678902626038, Validation Accuracy: 0.8082901835441589



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [8]:
from src.testing.testing import get_results, write_outputs

results = get_results(clf.predictions_proba, threshold, test_loader.dataset)
outputs = write_outputs(results, MODEL_DESCRIPTION, '/content/gdrive/My Drive/mcl-wic-outputs/OLD_LIBS_', validation_accuracy, threshold)